In [1]:
import json
import urllib
import urllib.request
import pandas as pd

from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.palettes import viridis, Category10, Category20, Turbo256
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.models.formatters import DatetimeTickFormatter
from datetime import datetime
from io import BytesIO

from decryptor import Decryptor

In [2]:
output_notebook()
pd.options.mode.copy_on_write = True

Loading BokehJS ...

In [3]:
BASE_URL = "https://tracking2024.vendeeglobe.org/data/"
CONFIG = "tracker_config"
REPORTS = "tracker_reports"
VERSION = datetime.now().strftime("%Y%m%d%H%M%S")

In [4]:

def download_and_decrypt(element):
    url = BASE_URL + element + ".hwx?version=" + VERSION
    request = urllib.request.Request(url,  headers={'User-Agent' : "Browser"})
    encrypted = urllib.request.urlopen(request).read()
    decrypted = Decryptor().decrypt(encrypted)
    return decrypted

In [5]:
XSLT="""<?xml version="1.0"?>
<xsl:stylesheet version="1.0" xmlns:xsl="http://www.w3.org/1999/XSL/Transform">

<xsl:template match="/">
<config>
    <xsl:for-each select="//boat">
    <boat>
            <id><xsl:value-of select="@id" /></id>
            <name><xsl:value-of select="@name"/></name>
            <skipper_fname><xsl:value-of select="./crew/navigator/@fname"/></skipper_fname>
            <skipper_lname><xsl:value-of select="./crew/navigator/@lname"/></skipper_lname>
            <skipper_photo><xsl:value-of select="./crew/navigator/@photo"/></skipper_photo>
    </boat>
    </xsl:for-each>
</config>
</xsl:template>
</xsl:stylesheet>
"""
config_xml = download_and_decrypt(CONFIG)
config = pd.read_xml(BytesIO(config_xml.encode("utf-8")), xpath="//boat", stylesheet=XSLT)

In [6]:
reports_json = download_and_decrypt(REPORTS)
r=json.loads(reports_json)
pr=pd.DataFrame(
    [[history_entry['date']] + line for history_entry in r['reports']['history'] for line in history_entry['lines']],
    columns=['date'] + r['reports']['columns'] 
)
pr.boat = pr.boat.astype(int)
pr.date = pd.to_datetime(pr.date, utc=True)
reports = pd.merge(pr, config, left_on='boat', right_on='id')

In [7]:
p = figure(width=800, height=600, toolbar_location=None)
boats=reports.query('date == date.max()').sort_values('rank')['boat'][0:20]
r=reports[reports['boat'].isin(boats)]
r['label'] = r['skipper_fname'] + " " + r['skipper_lname']
r['boat'] = pd.Categorical(r['boat'], categories=boats, ordered=True)
for (label, group), color in zip(r.groupby('label'), Category20[20]):
    p.line(x=group.date, y=group['dtl'], legend_label=str(label), color=color, line_width=2)

p.y_range.flipped = True
p.xaxis.formatter=DatetimeTickFormatter()
p.legend.location = "bottom_left"
show(p)

/Users/jmones/.pyenv/versions/3.9.15/lib/python3.9/site-packages/bokeh/util/serialization.py:243: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


In [8]:
p = figure(title="distance in 24h for the fastest skippers (mn)", width=1000, height=600, toolbar_location=None)
#r=reports.query("date>'2024-11-15T00:00:00Z'")
r=reports
r['label'] = r['skipper_fname'] + " " + r['skipper_lname']
labels=r.groupby('label')['dist24h'].max().sort_values(ascending=False).reset_index().head(15)['label']
r=r[r['label'].isin(labels)]
for (label, group), color in zip(r.groupby('label'), Category20[15]):
    p.line(x=group.date, y=group['dist24h'], legend_label=str(label), color=color, line_width=2)

p.xaxis.formatter=DatetimeTickFormatter()
p.legend.location = "bottom_left"

d = {item.label.value: item for item in p.legend.items}
p.legend.items = [ d[label] for label in labels]

show(p)


/Users/jmones/.pyenv/versions/3.9.15/lib/python3.9/site-packages/bokeh/util/serialization.py:243: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


In [9]:
p = figure(title="distance 4h for the fastest skippers (mn)", width=1000, height=600, toolbar_location=None)
#r=reports.query("date>'2024-11-15T00:00:00Z'")
r=reports
r.loc[:, 'label'] = r['skipper_fname'] + " " + r['skipper_lname']
labels=r.query('date == date.max()').sort_values('rank').reset_index().head(5)['label']
r=r[r['label'].isin(labels)]
r['datenotz']=r['date'].dt.tz_localize(None)
for (label, group), color in zip(r.groupby('label'), Category20[5]):
    p.line(x=group.datenotz, y=group['dist4h'], legend_label=str(label), color=color, line_width=2)

p.xaxis.formatter=DatetimeTickFormatter()
p.legend.location = "bottom_left"

d = {item.label.value: item for item in p.legend.items}
p.legend.items = [ d[label] for label in labels]

show(p)
